## # Introduction
<p><img src="https://i.imgur.com/kjWF1So.jpg" alt="Different characters on a computer screen"></p>
<p>According to a 2019 <a href="https://storage.googleapis.com/gweb-uniblog-publish-prod/documents/PasswordCheckup-HarrisPoll-InfographicFINAL.pdf">Google / Harris Poll</a>, 24% of Americans have used common passwords, like <code>abc123</code>, <code>Password</code>, and <code>Admin</code>. Even more concerning, 59% of Americans have incorporated personal information, such as their name or birthday, into their password. This makes it unsurprising that 4 in 10 Americans have had their personal information compromised online. Passwords with commonly used phrases and personal information makes cracking a password drastically easier.</p>
<p>You may have noticed over the years that password requirements have increased in complexity, including recommendations to change your passwords every couple of months. Compiled from industry recommendations, below is a list of passwords requirements you will be asked to test: </p>
<p><strong>Password Requirments:</strong></p>
<ol>
<li>Must be at least 10 characters in length</li>
<li>Must contain at least:<ul>
<li>one lower case letter </li>
<li>one upper case letter </li>
<li>one numeric character </li>
<li>one non-alphanumeric character</li></ul></li>
<li>Must not contain the phrase <code>password</code> (case insensitive)</li>
<li>Must not contain the user's first or last name, e.g., if the user's name is <code>John Smith</code>, then <code>SmItH876!</code> is not a valid password.</li>
</ol>
<p>Here is the dataset that you will investigate this project:</p>
<div style="background-color: #ebf4f7; color: #595959; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/logins.csv</b></div>
Each row represents a login credential. There are no missing values and you can consider the dataset "clean".
<ul>
    <li><b>id:</b> the user's unique ID.</li>
    <li><b>username:</b> the username with the format {firstname}.{lastname}.</li>
    <li><b>password:</b> the password that may or may not meet the requirements. <i>Note, passwords should never be saved in plaintext, always encrypt them when working with real live passwords!</i></li>
</ul>
</div>
<p>Warning: This dataset contains some <strong>real</strong> passwords leaked from <strong>real</strong> websites. These passwords have been filtered, but may still include words that are explicit and offensive.</p>
<p>From here on out, it will be your task to explore and manipulate the existing data until you can answer the two questions described in the instructions panel. Feel free to import as many packages as you need to complete your task, and add cells as necessary. Finally, remember that you are only tested on your answer, not on the methods you use to arrive at the answer!</p>
<p><strong>Note:</strong> To complete this project, you need to know how to manipulate strings in pandas DataFrames and be familiar with regular expressions. Before starting this project we recommend that you have completed the following courses: <a href="https://learn.datacamp.com/courses/data-cleaning-in-python">Data Cleaning in Python</a> and <a href="https://learn.datacamp.com/courses/regular-expressions-in-python">Regular Expressions in Python</a>.</p>

# Loading the data

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('datasets/logins.csv', index_col='id')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 982 entries, 1 to 982
Data columns (total 2 columns):
username    982 non-null object
password    982 non-null object
dtypes: object(2)
memory usage: 23.0+ KB


In [5]:
df.head()

,username,password
id,,
1,vance.jennings,vanceRules888!
2,consuelo.eaton,Mail_Pen%Scarlets.414
3,mitchel.perkins,Z00+1960
4,odessa.vaughan,D-rockyou
5,araceli.wilder,Araceli}r3


# Question 1. What percentage of users have invalid passwords?

Save your answer as a variable, bad_pass, in the form of a float rounded up to two decimals (e.g., 0.18).

First, let's analyze how many passwords break each individual requirement.

#### Requirement 1:

Must be at least 10 characters in length

In [6]:
too_short = df['password'].str.len() < 10

In [7]:
too_short.sum()

422

In [8]:
too_short.mean()

0.42973523421588594

#### Requirement 2:

Must contain at least:
 - one lower case letter
 - one upper case letter
 - one numeric character
 - one non-alphanumeric character

In [9]:
low_character_variety = ~df['password'].str.match(r'(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*\W).*')

In [10]:
low_character_variety.sum()

655

In [11]:
low_character_variety.mean()

0.6670061099796334

#### Requirement 3:
 
Must not contain the phrase password (case insensitive)

In [12]:
contains_password = df['password'].str.lower().str.contains('password')

In [13]:
contains_password.sum()

1

In [14]:
contains_password.mean()

0.0010183299389002036

#### Requirement 4:

Must not contain the user's first or last name, e.g., if the user's name is John Smith, then SmItH876! is not a valid password.

In [15]:
contains_users_name = df.apply(lambda r: any(s.lower() in r['password'].lower() for s in r['username'].split('.')), axis=1)

In [16]:
contains_users_name.sum()

19

In [17]:
contains_users_name.mean()

0.019348268839103868

#### Now for the final result:

In [18]:
bad_passwords = (too_short | low_character_variety | contains_password | contains_users_name)

In [19]:
bad_passwords.sum()

736

In [20]:
bad_passwords.mean()

0.7494908350305499

In [21]:
bad_pass = float(round(bad_passwords.mean(), 2))

# Question 2. Which users need to change their passwords?

Save your answer as a pandas Series consisting of the usernames in alphabetically ascending order called email_list. This will be used to automate email notifications to employees.

In [22]:
email_list = df[bad_passwords]['username'].sort_values()